In [75]:
import os, cv2
import numpy as np
import pandas as pd

# Paths
CSV_PATH = "NeoJaundice/chd_jaundice_published_2.csv"
IMG_DIR = "NeoJaundice/images"

# Load metadata
df = pd.read_csv(CSV_PATH)
images, labels = [], []

# Load and central crop
def crop_center(img, fraction=0.4):
    h, w = img.shape[:2]
    ch, cw = int(h * fraction), int(w * fraction)
    start_x, start_y = (w - cw) // 2, (h - ch) // 2
    return img[start_y:start_y+ch, start_x:start_x+cw]
def white_balance_with_yellow_patch(img, top_left=(90, 330), size=20):
    y, x = top_left
    patch = img[y:y+size, x:x+size]
    avg = np.mean(patch.reshape(-1, 3), axis=0)
    gain = np.array([0, 255, 255], dtype=np.float32) / (avg + 1e-6)
    balanced = np.clip(img.astype(np.float32) * gain, 0, 255).astype(np.uint8)
    return balanced


for _, row in df.iterrows():
    path = os.path.join(IMG_DIR, row['image_idx'])
    if os.path.exists(path):
        img = cv2.imread(path)
        img = cv2.resize(crop_center(img), (128, 128))
        images.append(img)
        labels.append(row['blood(mg/dL)'])

images = np.array(images)
labels = np.array(labels, dtype=np.float32)


In [76]:
rgb, hsv = [], []

for img in images:
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) / 255.0

    rgb_flat = rgb_img.reshape(-1, 3).T[:, :1223]  # (3, 1223)
    hsv_flat = hsv_img.reshape(-1, 3).T[:, :1223]  # (3, 1223)

    rgb.append(rgb_flat)
    hsv.append(hsv_flat)

rgb = np.array(rgb)  # (N, 3, 1223)
hsv = np.array(hsv)  # (N, 3, 1223)


In [77]:
from sklearn.model_selection import train_test_split

X_rgb_train, X_rgb_test, X_hsv_train, X_hsv_test, y_train, y_test = train_test_split(
    rgb, hsv, labels, test_size=0.2, random_state=42
)

In [91]:
from sklearn.preprocessing import StandardScaler

def standardize(X):
    N, C, T = X.shape
    X_flat = X.reshape(N, -1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_flat).reshape(N, C, T)
    return X_scaled, scaler

# Apply to both branches
X_rgb_train, rgb_scaler = standardize(X_rgb_train)
X_hsv_train, hsv_scaler = standardize(X_hsv_train)

X_rgb_test = rgb_scaler.transform(X_rgb_test.reshape(X_rgb_test.shape[0], -1)).reshape(-1, 3, 1223)
X_hsv_test = hsv_scaler.transform(X_hsv_test.reshape(X_hsv_test.shape[0], -1)).reshape(-1, 3, 1223)
# Swap axes
X_rgb_train = np.transpose(X_rgb_train, (0, 2, 1))  # (N, 1223, 3)
X_hsv_train = np.transpose(X_hsv_train, (0, 2, 1))
X_rgb_test = np.transpose(X_rgb_test, (0, 2, 1))
X_hsv_test = np.transpose(X_hsv_test, (0, 2, 1))


In [92]:
X_train = np.concatenate([X_rgb_train, X_hsv_train], axis=-1)
X_test  = np.concatenate([X_rgb_test, X_hsv_test], axis=-1)

In [93]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define single-branch model
input_layer = Input(shape=(1223, 6))  # Combined RGB+HSV

x = Conv1D(32, 3, activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2, padding='same')(x)

x = Conv1D(64, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2, padding='same')(x)

x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2, padding='same')(x)

x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2, padding='same')(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='linear')(x)

# Compile the model
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer=Adam(learning_rate=5e-4), loss='mse', metrics=['mae'])

model.summary()


Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_35 (InputLayer)       [(None, 1223, 6)]         0         
                                                                 
 conv1d_128 (Conv1D)         (None, 1223, 32)          608       
                                                                 
 batch_normalization_24 (Ba  (None, 1223, 32)          128       
 tchNormalization)                                               
                                                                 
 max_pooling1d_124 (MaxPool  (None, 612, 32)           0         
 ing1D)                                                          
                                                                 
 conv1d_129 (Conv1D)         (None, 612, 64)           6208      
                                                                 
 batch_normalization_25 (Ba  (None, 612, 64)           256

In [98]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
]

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=32,
    callbacks=callbacks
)



Epoch 1/150
45/45 [==============================] - 6s 110ms/step - loss: 42.8057 - mae: 5.1148 - val_loss: 97.8490 - val_mae: 8.5892 - lr: 5.0000e-04
Epoch 2/150
45/45 [==============================] - 5s 115ms/step - loss: 32.0928 - mae: 4.4954 - val_loss: 57.1456 - val_mae: 6.3661 - lr: 5.0000e-04
Epoch 3/150
45/45 [==============================] - 5s 117ms/step - loss: 27.2449 - mae: 4.1794 - val_loss: 37.5922 - val_mae: 5.0523 - lr: 5.0000e-04
Epoch 4/150
45/45 [==============================] - 4s 91ms/step - loss: 25.5491 - mae: 4.0445 - val_loss: 30.8961 - val_mae: 4.4554 - lr: 5.0000e-04
Epoch 5/150
45/45 [==============================] - 4s 85ms/step - loss: 24.5408 - mae: 3.9019 - val_loss: 30.7018 - val_mae: 4.4122 - lr: 5.0000e-04
Epoch 6/150
45/45 [==============================] - 4s 84ms/step - loss: 21.5666 - mae: 3.6454 - val_loss: 29.1792 - val_mae: 4.2914 - lr: 5.0000e-04
Epoch 7/150
45/45 [==============================] - 4s 85ms/step - loss: 22.4003 - mae: 3.

In [99]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

preds = model.predict([X_test])
rmse = np.sqrt(mean_squared_error(y_test, preds))
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print(f"\nFinal Results\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}\nR²: {r2:.2f}")


14/14 [==============================] - 0s 13ms/step

Final Results
RMSE: 4.01
MAE: 3.05
R²: 0.45
